In [29]:
import os

import pandas as pd
import json


# Table conversion

In [30]:
def read_json(json_file_path: str):
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")
        return
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {json_file_path}")
        return

In [31]:
INPUT_JSON_FILE_PATH = os.path.join("data", "01_raw", "octopart.json")
OUTPUT_JSON_FILE_PATH = os.path.join("data", "02_processed", "octopart.json")

raw_json = read_json(INPUT_JSON_FILE_PATH)

result_json_list = raw_json['data']['supSearchMpn']['results'][0]['part']['similarParts']

In [35]:
def extract_top_suppliers(part_data, top_n=5):
    """
    Extract top N suppliers for a single part based on unit price.
    
    Args:
        part_data (dict): Single part data dictionary
        top_n (int): Number of top suppliers to return
        
    Returns:
        pd.DataFrame: DataFrame with top suppliers for this part
    """
    if not part_data.get('sellers'):
        return pd.DataFrame()
    
    # Load sellers into DataFrame
    sellers_df = pd.DataFrame(part_data['sellers'])
    
    if sellers_df.empty:
        return pd.DataFrame()
    
    # Extract company info early
    sellers_df['company_id'] = sellers_df['company'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
    sellers_df['company_name'] = sellers_df['company'].apply(lambda x: x.get('name') if isinstance(x, dict) else None)
    sellers_df.drop(columns='company', inplace=True)
    
    # Ensure offers is a list and explode
    sellers_df['offers'] = sellers_df['offers'].apply(lambda x: x if isinstance(x, list) else [])
    sellers_df = sellers_df.explode('offers')
    
    # Clean offers and extract prices
    sellers_df['offers'] = sellers_df['offers'].apply(lambda x: x if isinstance(x, dict) else {})
    sellers_df['prices'] = sellers_df['offers'].apply(lambda x: x.get('prices', []))
    sellers_df.drop(columns='offers', inplace=True)
    
    # Explode prices and normalize
    sellers_df = sellers_df.explode('prices')
    sellers_df['prices'] = sellers_df['prices'].apply(lambda x: x if isinstance(x, dict) else {})
    sellers_df = pd.concat([sellers_df.drop(columns='prices'), sellers_df['prices'].apply(pd.Series)], axis=1)
    
    # Filter invalid or zero prices
    sellers_df = sellers_df[sellers_df['convertedPrice'].notnull() & (sellers_df['convertedPrice'] != 0)]
    
    if sellers_df.empty:
        return pd.DataFrame()
    
    # Rename and reorder columns
    sellers_df.rename(columns={'quantity': 'min_quantity', 'convertedPrice': 'unit_price'}, inplace=True)
    
    # Select and reorder columns (handle missing columns gracefully)
    available_columns = ['country', 'company_id', 'company_name', 'min_quantity', 'unit_price', 'convertedCurrency']
    existing_columns = [col for col in available_columns if col in sellers_df.columns]
    sellers_df = sellers_df[existing_columns]
    
    # Reset index and sort by price
    sellers_df.reset_index(drop=True, inplace=True)
    sellers_df.sort_values(by='unit_price', inplace=True)
    
    # Return top N suppliers
    return sellers_df.iloc[:top_n].copy()

def extract_descriptions(part_data):
    """
    Extract and join descriptions from part data.
    
    Args:
        part_data (dict): Single part data dictionary
        
    Returns:
        str: Joined descriptions with spaces
    """
    descriptions = part_data.get('descriptions', [])
    if not descriptions:
        return ''
    
    # Extract text from each description and join with spaces
    description_texts = []
    for desc in descriptions:
        if isinstance(desc, dict) and 'text' in desc:
            description_texts.append(desc['text'])
    
    return ' '.join(description_texts)

def process_parts_list_with_suppliers(raw_json, top_suppliers=5):
    """
    Process a list of part data dictionaries into DataFrames with supplier information.
    
    Args:
        raw_json (dict): Raw JSON data containing parts information
        top_suppliers (int): Number of top suppliers to include per part
        
    Returns:
        tuple: (parts_df, suppliers_df) - Parts DataFrame and combined suppliers DataFrame
    """
    # Extract the parts list
    result_json_list = raw_json['data']['supSearchMpn']['results'][0]['part']['similarParts']
    
    all_parts_data = []
    all_suppliers_data = []
    
    for idx, part_data in enumerate(result_json_list):
        
        # Extract basic part information
        part_info = {
            'part_id': idx,  # Add unique identifier for joining
            'name': part_data.get('name', ''),
            'mpn': part_data.get('mpn', ''),
            'manufacturer': part_data.get('manufacturer', {}).get('name', ''),
            'short_description': part_data.get('shortDescription', ''),
            'category': part_data.get('category', {}).get('name', ''),
            'descriptions': extract_descriptions(part_data),  # Add joined descriptions
            'total_sellers': len(set(seller.get('company', {}).get('name', 'Unknown') 
                                    for seller in part_data.get('sellers', []) 
                                    if seller.get('company', {}).get('name')))  # Add unique sellers by company name
        }
        
        # Find best seller (most economic option) - keeping original logic
        best_seller_info = {
            'best_seller_company': 'No sellers available',
            'best_seller_country': '',
            'best_unit_price': '',
            'best_price_currency': '',
            'best_price_min_qty': ''
        }
        
        lowest_price = float('inf')
        
        for seller in part_data.get('sellers', []):
            company_name = seller.get('company', {}).get('name', 'Unknown')
            country = seller.get('country', 'Unknown')
            
            for offer in seller.get('offers', []):
                prices = offer.get('prices', [])
                
                if prices:
                    for price_break in prices:
                        unit_price = price_break.get('convertedPrice', price_break.get('convertedPrice', float('inf')))
                        quantity = price_break.get('quantity', 1)
                        currency = price_break.get('convertedCurrency', price_break.get('convertedCurrency', 'USD'))
                        
                        if unit_price < lowest_price:
                            lowest_price = unit_price
                            best_seller_info = {
                                'best_seller_company': company_name,
                                'best_seller_country': country,
                                'best_unit_price': f"{unit_price:.4f}",
                                'best_price_currency': currency,
                                'best_price_min_qty': quantity
                            }
        
        # Combine part info with best seller info
        combined_info = {**part_info, **best_seller_info}
        
        # Add specs as columns
        for spec in part_data.get('specs', []):
            column_name = spec['attribute']['name'].replace(' ', '_').replace('(', '').replace(')', '').lower()
            value_with_units = f"{spec['value']} {spec['units']}".strip()
            combined_info[column_name] = value_with_units
        
        all_parts_data.append(combined_info)
        
        # Extract top suppliers for this part
        suppliers_df = extract_top_suppliers(part_data, top_suppliers)
        if not suppliers_df.empty:
            # Add part identifier to suppliers
            suppliers_df['part_id'] = idx
            suppliers_df['part_name'] = part_data.get('name', '')
            suppliers_df['part_mpn'] = part_data.get('mpn', '')
            
            # Add supplier rank
            suppliers_df['supplier_rank'] = range(1, len(suppliers_df) + 1)
            
            all_suppliers_data.append(suppliers_df)
    
    # Create parts DataFrame
    parts_df = pd.DataFrame(all_parts_data)
    nan_percentage = parts_df.isnull().sum() / len(parts_df)
    
    # Keep columns where NaN percentage < 0.5 (50%)
    final_parts_df = parts_df.loc[:, nan_percentage < 0.5]
    
    # Create combined suppliers DataFrame
    if all_suppliers_data:
        final_suppliers_df = pd.concat(all_suppliers_data, ignore_index=True)
        
        # Reorder columns for better readability
        supplier_columns = ['part_id', 'part_name', 'part_mpn', 'supplier_rank', 'company_name', 
                          'country', 'unit_price', 'convertedCurrency', 'min_quantity', 'company_id']
        existing_supplier_columns = [col for col in supplier_columns if col in final_suppliers_df.columns]
        final_suppliers_df = final_suppliers_df[existing_supplier_columns]
    else:
        final_suppliers_df = pd.DataFrame()
    
    return final_parts_df, final_suppliers_df

def create_combined_parts_suppliers_view(parts_df, suppliers_df):
    """
    Create a combined view with parts and their top suppliers in separate columns.
    
    Args:
        parts_df (pd.DataFrame): Parts DataFrame
        suppliers_df (pd.DataFrame): Suppliers DataFrame
        
    Returns:
        pd.DataFrame: Combined DataFrame with supplier info as additional columns
    """
    if suppliers_df.empty:
        return parts_df
    
    # Pivot suppliers to have top N as columns
    supplier_pivot = suppliers_df.pivot_table(
        index='part_id', 
        columns='supplier_rank',
        values=['company_name', 'country', 'unit_price', 'convertedCurrency', 'min_quantity'],
        aggfunc='first'
    )
    
    # Flatten column names
    supplier_pivot.columns = [f'{col[0]}_supplier_{col[1]}' for col in supplier_pivot.columns]
    supplier_pivot.reset_index(inplace=True)
    
    # Merge with parts DataFrame
    combined_df = parts_df.merge(supplier_pivot, on='part_id', how='left')
    
    return combined_df.to_markdown()

In [36]:
parts_df, suppliers_df = process_parts_list_with_suppliers(raw_json, top_suppliers=3)
combined_view = create_combined_parts_suppliers_view(parts_df, suppliers_df)

In [37]:
print(combined_view)

|    |   part_id | name                          | mpn                  | manufacturer   | short_description   | category           | descriptions                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |   total_sellers | best_seller_company   | best_seller_country   |   best_unit_price | best_price_currency   |   best_price_min_qty | capacitance   |   case/package |   case_code_imperial |   case_code_metric | depth   | dielectric   | height   | lead_free   | length    | material   | max_operating_temperature   | mi